In [2]:
import tensorflow as tf
import cv2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import PIL

In [5]:
from tensorflow.keras import layers
import pickle
from tensorflow.python.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler

In [11]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()


In [14]:
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.1,shuffle=True)

In [106]:
x_train=x_train/255
x_val=x_val/255
x_test=x_test/255

In [107]:
print(x_train.shape,x_val.shape,x_test.shape,y_train.shape,y_test.shape)

(40500, 32, 32, 3) (4500, 32, 32, 3) (10000, 32, 32, 3) (40500, 1) (10000, 1)


### Building and Training Residual Model

In [108]:
def resblock(X,filter,stage):
    #convolutional Block
    X_copy=X
    f1,f2,f3=filter
    # Main Path
    X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_conv_a', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = MaxPool2D((2,2))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_a')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_conv_b', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_b')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_c', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_c')(X)


    # Short path
    X_copy = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_copy', kernel_initializer= glorot_uniform(seed = 0))(X_copy)
    X_copy = MaxPool2D((2,2))(X_copy)
    X_copy = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_copy')(X_copy)

    # ADD
    X = Add()([X,X_copy])
    X = Activation('relu')(X)

    # Identity Block 1
    X_copy = X


    # Main Path
    X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_1_a', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_a')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_1_b', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_b')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_1_c', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_c')(X)

    # ADD
    X = Add()([X,X_copy])
    X = Activation('relu')(X)

    # Identity Block 2
    X_copy = X


    # Main Path
    X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_2_a', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_a')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_2_b', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_b')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_2_c', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_c')(X)

    # ADD
    X = Add()([X,X_copy])
    X = Activation('relu')(X)

    return X

In [121]:
input_shape = (32, 32, 3)

# Input tensor shape
X_input = Input(input_shape)

# Zero-padding
X = ZeroPadding2D((3, 3))(X_input)

# 1 - stage
X = Conv2D(64, (7, 7), strides= (2, 2), name = 'conv1', kernel_initializer= glorot_uniform(seed = 0))(X)
X = BatchNormalization(axis =3, name = 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3, 3), strides= (2, 2))(X)

# 2 - stage
X = resblock(X, filter= [64, 64, 256], stage= 2)

# 3 - stage
X = resblock(X, filter= [128, 128, 512], stage= 3)

# Average Pooling
X = AveragePooling2D((1, 1), name = 'Averagea_Pooling')(X)

# Final layer
X = Flatten()(X)
X = Dense(10, activation = 'softmax', name = 'Dense_final', kernel_initializer= glorot_uniform(seed=0))(X)

model_ = Model( inputs= X_input, outputs = X, name = 'Custom Resnet')

model_.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_24 (ZeroPadding2 (None, 38, 38, 3)    0           input_25[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        zero_padding2d_24[0][0]          
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [122]:
# Training The Network
model_.compile(optimizer = "Adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [123]:
# Using early stopping to exit training if validation loss is not decreasing even after certain epochs (patience)
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 20)

# Saving the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath = "Cifar10.hdf5", verbose = 1, save_best_only=True)

In [126]:
from tensorflow.python.keras.utils import to_categorical

y_train_encoded = to_categorical(y_train)
y_val_encoded=to_categorical(y_val)
y_test_encoded = to_categorical(y_test)

In [ ]:
history = model_.fit(x_train,y_train_encoded,
          validation_data=(x_val, y_val_encoded), steps_per_epoch=len(x_train) // 64,
          epochs= 20, callbacks=[checkpointer, earlystopping])

Epoch 1/20


## Normal Conv

In [95]:
from tensorflow.python.keras.utils import to_categorical

y_train_encoded = to_categorical(y_train)
y_val_encoded=to_categorical(y_val)
y_test_encoded = to_categorical(y_test)


In [96]:
model=tf.keras.models.Sequential([
    Conv2D(16,(3,3),activation='relu', input_shape=(32,32,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(48,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(10,activation='sigmoid')
])

In [88]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 30, 30, 16)        448       
_________________________________________________________________
max_pooling2d_68 (MaxPooling (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 13, 13, 32)        4640      
_________________________________________________________________
max_pooling2d_69 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 4, 4, 48)          13872     
_________________________________________________________________
max_pooling2d_70 (MaxPooling (None, 2, 2, 48)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 192)               0         
__________

In [99]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=.001),metrics=['accuracy'])

In [100]:
nx_train=x_train/255
nx_val=x_val/255
nx_test=x_test/255

In [101]:
print(nx_val.shape,y_val_encoded.shape)

(4500, 32, 32, 3) (4500, 10)


In [104]:
history=model.fit(nx_train,y_train_encoded,steps_per_epoch=8
                 ,epochs=10,verbose=1,validation_data=(nx_val,y_val_encoded),
                 validation_steps=8)

Epoch 1/10
4/8 [==============>...............] - ETA: 2:08 - loss: 0.5185 - acc: 0.9000

KeyboardInterrupt: 